In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#install openpyxl, needed for exporting datasets to excel-file.

!pip install openpyxl

/kaggle/input/grid100globalpredictorsutrecht/grid100_GlobalPredictors-Utrecht-processed.csv
/kaggle/input/datasetlowestaicspatialglobal/dataset_lowestAIC_longlat.csv
/kaggle/input/grid100globalpredictorshh/grid100_GlobalPredictors-HH_processed.csv
/kaggle/input/grid100globalpredictorsbayreuth/grid100_GlobalPredictors-Bayreuth_processed.csv
/kaggle/input/grid100globalpredictors/grid100_GlobalPredictors-processed.csv
     |████████████████████████████████| 242 kB 5.2 MB/s            


## import train- and test datasets

In [2]:
#import training dataset
df_train = pd.read_csv(r'../input/datasetlowestaicspatialglobal/dataset_lowestAIC_longlat.csv', sep=';')

#AMSTERDAM
#df_test_xy = pd.read_csv(r'../input/grid100globalpredictors/grid100_GlobalPredictors-processed.csv', sep=';')

#HAMBURG
#df_test_xy = pd.read_csv(r'../input/grid100globalpredictorshh/grid100_GlobalPredictors-HH_processed.csv', sep=';')

#Bayreuth
#df_test_xy = pd.read_csv(r'../input/grid100globalpredictorsbayreuth/grid100_GlobalPredictors-Bayreuth_processed.csv', sep=';')

#Utrecht
df_test_xy = pd.read_csv(r'../input/grid100globalpredictorsutrecht/grid100_GlobalPredictors-Utrecht-processed.csv', sep=';')

In [3]:
#replace NA with 0
df_train=df_train.fillna(0)

In [4]:
#examine
df_test_xy.head()

,nightlight_450,nightlight_3150,population_1000,population_3000,road_class_2_25,road_class_3_3000,road_class_3_300,trop_mean_filt,BldDen100,NDVI,trafBuf25,trafBuf50,Longitude,Latitude
0,1.346078,0.719939,502.881683,2435.422607,0.0,15927.90625,904.727905,0.000079,0.310619,5962,0.0,0.0,4.901548,51.902208
1,0.732517,0.711915,479.631622,2492.685059,0.0,15861.02637,840.276916,0.000079,0.201087,5962,0.0,0.0,4.902998,51.902258
2,0.732517,0.711915,479.865601,2492.246094,0.0,15790.44922,495.902496,0.000079,0.052682,5962,0.0,0.0,4.904447,51.902309
3,0.732517,0.711915,393.215485,2592.952148,0.0,15696.45996,398.601898,0.000079,0.000000,6486,0.0,0.0,4.905897,51.902359
4,0.732517,0.711915,393.390961,2592.251709,0.0,15618.36035,240.843781,0.000079,0.000000,6486,0.0,0.0,4.907347,51.902410


In [5]:

#remove unnecessary information in training dataset
df_train = df_train.drop(['Longitude', 'Latitude'], axis=1)

#import testing dataset - to this unprocessed dataset, the predicted NO2 will eventually be added.

#HAMBURG
# df_test_xy = pd.read_csv(r'../input/predictorshh/predictorsHH.csv', sep=';')

#UTRECHT
# df_test_xy = pd.read_csv(r'../input/predictorsutrecht-csv/predictorsUtrecht.csv', sep=';')

#NIJMEGEN
# df_test_xy = pd.read_csv(r'../input/predictorsnijmegen-csv/predictorsNijmegen.csv', sep=';')

# #BAYREUTH
# df_test_xy = pd.read_csv(r'../input/predictorshamburgdecemberdata-csv/predictorsHamburg.csv', sep=';')

# #AMSTERDAM 25M 
# df_test_xy = pd.read_csv(r'../input/predictorsamsterdam-25m/predictorsAmsterdam_25m.csv', sep=';')


#remove unnecessary information in testing dataset
df_test_proc = df_test_xy.drop(['Longitude', 'Latitude'], axis=1)

#define response- and predictor variables
Y_train = df_train['mean_value_NO2']
X_train = df_train.drop(['mean_value_NO2'], axis=1)
X_test = df_test_proc

#validate if columns of training and testing dataset are the same
print(len(X_test.columns))
print(len(X_train.columns))

12
12


### drop columns

In [6]:
X_train.columns

Index(['nightlight_450', 'nightlight_3150', 'population_1000',
       'population_3000', 'road_class_2_25', 'road_class_3_3000',
       'road_class_3_300', 'trop_mean_filt_2019', 'BldDen100', 'NDVI',
       'trafBuf25', 'trafBuf50'],
      dtype='object')

In [7]:
X_test.columns

Index(['nightlight_450', 'nightlight_3150', 'population_1000',
       'population_3000', 'road_class_2_25', 'road_class_3_3000',
       'road_class_3_300', 'trop_mean_filt', 'BldDen100', 'NDVI', 'trafBuf25',
       'trafBuf50'],
      dtype='object')

In [8]:
# X_train = X_train.drop(['population_1000'], axis=1)
# X_test = X_test.drop(['population_1000'], axis=1)

In [9]:
df_test_xy.head()

,nightlight_450,nightlight_3150,population_1000,population_3000,road_class_2_25,road_class_3_3000,road_class_3_300,trop_mean_filt,BldDen100,NDVI,trafBuf25,trafBuf50,Longitude,Latitude
0,1.346078,0.719939,502.881683,2435.422607,0.0,15927.90625,904.727905,0.000079,0.310619,5962,0.0,0.0,4.901548,51.902208
1,0.732517,0.711915,479.631622,2492.685059,0.0,15861.02637,840.276916,0.000079,0.201087,5962,0.0,0.0,4.902998,51.902258
2,0.732517,0.711915,479.865601,2492.246094,0.0,15790.44922,495.902496,0.000079,0.052682,5962,0.0,0.0,4.904447,51.902309
3,0.732517,0.711915,393.215485,2592.952148,0.0,15696.45996,398.601898,0.000079,0.000000,6486,0.0,0.0,4.905897,51.902359
4,0.732517,0.711915,393.390961,2592.251709,0.0,15618.36035,240.843781,0.000079,0.000000,6486,0.0,0.0,4.907347,51.902410


In [10]:
# specify city so that it is included in output-name.
city = "Utrecht100"

In [11]:
X_train.head()

,nightlight_450,nightlight_3150,population_1000,population_3000,road_class_2_25,road_class_3_3000,road_class_3_300,trop_mean_filt_2019,BldDen100,NDVI,trafBuf25,trafBuf50
0,22.585377,13.402575,8123.319336,54316.64453,0.000000,203502.42190,4167.064941,0.000058,1.000663,2310.0,0.000000,0.000000
1,23.546194,22.862799,9558.207031,76786.28125,0.000000,99596.04688,1850.278076,0.000081,0.978245,2518.0,0.000000,0.000000
2,18.290140,16.493486,10471.541020,88439.60938,46.359188,265563.71880,2016.838379,0.000076,0.914638,3216.0,674.791667,674.791667
3,23.645823,18.248226,11165.945310,76961.00000,0.000000,97921.56250,478.363281,0.000074,0.712983,1052.0,0.000000,0.000000
4,14.026642,5.514785,6017.511230,30814.58984,0.000000,176265.68750,3338.221191,0.000047,1.000641,3240.0,0.000000,0.000000


In [12]:
X_test.head()

,nightlight_450,nightlight_3150,population_1000,population_3000,road_class_2_25,road_class_3_3000,road_class_3_300,trop_mean_filt,BldDen100,NDVI,trafBuf25,trafBuf50
0,1.346078,0.719939,502.881683,2435.422607,0.0,15927.90625,904.727905,0.000079,0.310619,5962,0.0,0.0
1,0.732517,0.711915,479.631622,2492.685059,0.0,15861.02637,840.276916,0.000079,0.201087,5962,0.0,0.0
2,0.732517,0.711915,479.865601,2492.246094,0.0,15790.44922,495.902496,0.000079,0.052682,5962,0.0,0.0
3,0.732517,0.711915,393.215485,2592.952148,0.0,15696.45996,398.601898,0.000079,0.000000,6486,0.0,0.0
4,0.732517,0.711915,393.390961,2592.251709,0.0,15618.36035,240.843781,0.000079,0.000000,6486,0.0,0.0


In [13]:
# X_test['BD_100m'] = X_test['BD_100m'].replace(np.nan, 0)

In [14]:
# X_test['trop_mean_filt'] = [float(str(i).replace("c(", "")) for i in X_test["trop_mean_filt"]]

In [15]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88752 entries, 0 to 88751
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   nightlight_450     88752 non-null  float64
 1   nightlight_3150    88752 non-null  float64
 2   population_1000    88752 non-null  float64
 3   population_3000    88752 non-null  float64
 4   road_class_2_25    88752 non-null  float64
 5   road_class_3_3000  88752 non-null  float64
 6   road_class_3_300   88752 non-null  float64
 7   trop_mean_filt     88752 non-null  float64
 8   BldDen100          88752 non-null  float64
 9   NDVI               88752 non-null  int64  
 10  trafBuf25          88752 non-null  float64
 11  trafBuf50          88752 non-null  float64
dtypes: float64(11), int64(1)
memory usage: 8.1 MB


In [16]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482 entries, 0 to 481
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   nightlight_450       482 non-null    float64
 1   nightlight_3150      482 non-null    float64
 2   population_1000      482 non-null    float64
 3   population_3000      482 non-null    float64
 4   road_class_2_25      482 non-null    float64
 5   road_class_3_3000    482 non-null    float64
 6   road_class_3_300     482 non-null    float64
 7   trop_mean_filt_2019  482 non-null    float64
 8   BldDen100            482 non-null    float64
 9   NDVI                 482 non-null    float64
 10  trafBuf25            482 non-null    float64
 11  trafBuf50            482 non-null    float64
dtypes: float64(12)
memory usage: 45.3 KB


In [17]:
X_train.describe()

,nightlight_450,nightlight_3150,population_1000,population_3000,road_class_2_25,road_class_3_3000,road_class_3_300,trop_mean_filt_2019,BldDen100,NDVI,trafBuf25,trafBuf50
count,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000
mean,15.339811,11.042987,6154.486286,41489.437120,14.572097,136692.065999,2314.033584,0.000067,0.677052,3289.904564,128.701763,146.889530
std,13.528546,10.940724,4509.973126,34979.658532,34.954581,83339.859648,1547.899665,0.000025,0.375196,1439.762627,432.308261,452.133783
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000026,0.000000,0.000000,0.000000,0.000000
25%,4.616748,2.898382,2204.519532,11452.700443,0.000000,70823.226567,930.852341,0.000047,0.401915,2248.000000,0.000000,0.000000
50%,14.009659,8.198158,5945.538330,33821.939455,0.000000,134524.296900,2447.691162,0.000059,0.882949,3122.000000,0.000000,0.000000
75%,22.395457,16.505301,9036.756348,61824.045898,0.000000,193091.816425,3403.388611,0.000084,0.992667,4173.000000,0.000000,0.000000
max,84.318871,100.999016,20300.886720,165271.375000,164.927063,444277.312500,7239.326660,0.000143,1.000800,7775.000000,5112.958333,5112.958333


In [18]:
X_test.describe()

,nightlight_450,nightlight_3150,population_1000,population_3000,road_class_2_25,road_class_3_3000,road_class_3_300,trop_mean_filt,BldDen100,NDVI,trafBuf25,trafBuf50
count,88752.000000,88752.000000,88752.000000,88752.000000,88752.000000,88752.000000,88752.000000,88752.000000,88752.000000,88752.000000,88752.000000,88752.000000
mean,5.363768,5.378962,2188.744906,19751.988188,0.830072,73783.467933,721.606252,0.000079,0.203116,4663.679275,19.635520,34.318195
std,7.793616,5.630151,3132.723658,21765.126856,6.919148,64816.386222,1292.594491,0.000003,0.337001,1794.603421,194.079841,253.680270
min,0.000000,0.386488,0.000000,1561.082031,0.000000,3307.272461,0.000000,0.000073,0.000000,-14.000000,0.000000,0.000000
25%,0.720875,1.411368,289.775635,4973.127930,0.000000,25805.644528,0.000000,0.000077,0.000000,3422.000000,0.000000,0.000000
50%,2.022838,3.243662,746.741821,10810.468750,0.000000,49191.267575,7.872908,0.000078,0.000000,5009.000000,0.000000,0.000000
75%,6.603918,7.011188,2589.979309,26339.443360,0.000000,106230.501925,687.598373,0.000081,0.278501,6003.000000,0.000000,0.000000
max,87.565384,27.272654,15167.037110,113054.562500,190.947022,358706.312500,8519.986328,0.000089,0.999543,9798.000000,4168.166667,4168.166667


## model 1 - Random Forest

In [19]:
## PARAMETERS FOR MODEL - RANDOM FOREST
#import necessary module(s) for random forest
from sklearn.ensemble import RandomForestRegressor
# # Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42, min_samples_split=10,
                          min_samples_leaf=5,max_features=4,max_depth=10, bootstrap=True )
# Train the model on training data
rf.fit(X_train, Y_train)

#predict NO2 values, based on random forest model that is trained by the training dataset
predicted_NO2 = rf.predict(X_test)

#add predicted values as a column to copy of original dataset ("df_test_xy")
df_test_xy_RF = df_test_xy
df_test_xy_RF['predicted_NO2_RF'] = predicted_NO2

#export to .xlsx & .csv
df_test_xy_RF.to_excel('Predicting NO2-RF'+ str(city)+'_xy.xlsx')
df_test_xy_RF.to_csv('Predicting NO2-RF'+ str(city)+'_xy.csv')

## model 2 - LASSO

In [20]:
#import necessary module(s) for LASSO
from sklearn.linear_model import Lasso

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#standardizing is necessary 
# X_train_stan = sc.fit_transform(X_train)
# X_test_stan = sc.fit_transform(X_test)

#set alpha to 0.1 and train model based on training data
model_lasso = Lasso(alpha=0.1)
model_lasso.fit(X_train, Y_train)

#predict NO2 values, based on LASSO model that is trained by the training dataset
predicted_NO2= model_lasso.predict(X_test)

#add predicted values as a column to copy of the original dataset ("df_test_xy")
df_test_xy_LAS = df_test_xy
df_test_xy_LAS['predicted_NO2_LASSO'] = predicted_NO2

#export to .xlsx & .csv
df_test_xy_LAS.to_excel('Predicting NO2-RF-LAS-'+ str(city)+'_xy.xlsx')
df_test_xy_LAS.to_csv('Predicting NO2-RF-LAS-'+ str(city)+'_xy.csv')

In [21]:
print(df_test_xy_LAS)

       nightlight_450  nightlight_3150  population_1000  population_3000  \
0            1.346078         0.719939       502.881683      2435.422607   
1            0.732517         0.711915       479.631622      2492.685059   
2            0.732517         0.711915       479.865601      2492.246094   
3            0.732517         0.711915       393.215485      2592.952148   
4            0.732517         0.711915       393.390961      2592.251709   
...               ...              ...              ...              ...   
88747        4.986944         4.660944      1666.839966     11873.924800   
88748        4.986944         4.660944      1664.778687     11869.287110   
88749        4.986944         4.660944      1703.642090     11952.334960   
88750        7.999596         4.564402      1924.614990     12425.597660   
88751        8.316184         4.591713      2018.294434     12641.467770   

       road_class_2_25  road_class_3_3000  road_class_3_300  trop_mean_filt  \
0       

## model 3 - RIDGE

In [22]:
#import necessary module(s) for RIDGE
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV

#set alpha to 0.3 and train model based on training data
model_ridge = Ridge(alpha = 0.3, normalize = True)
model_ridge.fit(X_train, Y_train) #again, data is standardized

#predict NO2 values, based on RIDGE model that is trained by the training dataset
predicted_NO2= model_ridge.predict(X_test)

#add predicted values as a column to copy of the original dataset ("df_test_xy")
df_test_xy_RID = df_test_xy
df_test_xy_RID['predicted_NO2_RIDGE'] = predicted_NO2

#export to .xlsx & .csv
df_test_xy_RID.to_excel('Predicting NO2-RF-LAS-RID-'+ str(city)+'_xy.xlsx')
df_test_xy_RID.to_csv('Predicting NO2-RF-LAS-RID-'+ str(city)+'_xy.csv')

## model 4 - LightGBM

In [23]:
#import necessary module(s) for LightGBM
import lightgbm as lgb

#set model parameters and train model on training dataset
model = lgb.LGBMRegressor(reg_alpha =2, reg_lambda = 0, max_depth = 5, learning_rate = 0.0005, n_estimators =50000, random_state=42)
model.fit(X_train, Y_train)

#predict NO2 values, based on LightGBM model that is trained by the training dataset
predicted_NO2 = model.predict(X_test)

#add predicted values as a column to copy of the original dataset ("df_test_xy")
df_test_xy_LGB = df_test_xy
df_test_xy_LGB['predicted_NO2_LightGBM'] = predicted_NO2

#export to .xlsx & .csv
df_test_xy_LGB.to_excel('Predicting NO2-RF-LAS-RID-LGB-'+ str(city)+'_xy.xlsx')
df_test_xy_LGB.to_csv('Predicting NO2-RF-LAS-RID-LGB-'+ str(city)+'_xy.csv')

## model 5A - XGBoost

In [24]:
#import necessary module(s) for XGBoost
import xgboost as xgb

#set model parameters and train model on training dataset
xg_reg = xgb.XGBRegressor(gamma = 5,  reg_alpha =2, reg_lambda=0, max_depth = 5, learning_rate = 0.0005, n_estimators=50000, random_state=42)
xg_reg.fit(X_train, Y_train)

#predict NO2 values, based on XGBoost model that is trained by the training dataset
predicted_NO2 = xg_reg.predict(X_test)

#add predicted values as a column to copy of the original dataset ("df_test_xy")
df_test_xy_XGB = df_test_xy
df_test_xy_XGB['predicted_NO2_XGBoost'] = predicted_NO2

#export to .xlsx & .csv
df_test_xy_XGB.to_excel('Predicting NO2-AllModels'+ str(city)+'_xy.xlsx')
df_test_xy_XGB.to_csv('Predicting NO2-AllModels'+ str(city)+'_xy.csv')

In [25]:
print(df_test_xy_XGB)

       nightlight_450  nightlight_3150  population_1000  population_3000  \
0            1.346078         0.719939       502.881683      2435.422607   
1            0.732517         0.711915       479.631622      2492.685059   
2            0.732517         0.711915       479.865601      2492.246094   
3            0.732517         0.711915       393.215485      2592.952148   
4            0.732517         0.711915       393.390961      2592.251709   
...               ...              ...              ...              ...   
88747        4.986944         4.660944      1666.839966     11873.924800   
88748        4.986944         4.660944      1664.778687     11869.287110   
88749        4.986944         4.660944      1703.642090     11952.334960   
88750        7.999596         4.564402      1924.614990     12425.597660   
88751        8.316184         4.591713      2018.294434     12641.467770   

       road_class_2_25  road_class_3_3000  road_class_3_300  trop_mean_filt  \
0       